In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import urllib.request, json 
#from tensorflow import keras

import pandas as pd
import json
import numpy as np
import os
import time

device = None
LongTensor = None
if torch.cuda.is_available(): # device agnostic code
    device = torch.device('cuda')
    LongTensor = torch.cuda.LongTensor
    print("using gpu")
else:                                                   #
    device = torch.device('cpu')
    LongTensor = torch.LongTensor
    print("using cpu")


using cpu


In [2]:
# Dataset Preparation
print ("Read Dataset ... ")
def read_dataset(path):
	return json.load(open(path)) 

def read_dataset_url(path):
  with urllib.request.urlopen(path) as url:
    data = json.load(url)#.read().decode())
    return data

try:
    train = read_dataset('../data/kaggle_cooking/train.json')
    test = read_dataset('../data/kaggle_cooking/test.json')
except:
    train = read_dataset_url('https://raw.githubusercontent.com/AssassinTee/Seminar/master/data/kaggle_cooking/train.json')
    test = read_dataset_url('https://raw.githubusercontent.com/AssassinTee/Seminar/master/data/kaggle_cooking/test.json')
    

Read Dataset ... 


In [4]:
print ("Prepare text data of Train and Test ... ")
def generate_text(data):
	text_data = [" ".join(doc['ingredients']).lower() for doc in data]
	return text_data

train_text = generate_text(train)
test_text = generate_text(test)
target = [doc['cuisine'] for doc in train]
#print(target)
#print(len(target))
target_set=set(target)
#print(target_set)
#print(len(target_set))

Prepare text data of Train and Test ... 


In [5]:
tfidf = TfidfVectorizer(binary=True)
def tfidf_features(txt, flag):
    if flag == "train":
        x = tfidf.fit_transform(txt)
    else:
        x = tfidf.transform(txt)
    x = x.astype('float16')
    return x

X = tfidf_features(train_text, flag="train")
X_test = tfidf_features(test_text, flag="test")

In [6]:
lb = LabelEncoder()
ylab = lb.fit_transform(target)
ylab = ylab.reshape((len(ylab), 1))

In [7]:
#Init Model Weights
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
        
#Build Sequential Model        
model = nn.Sequential(nn.Linear(3010, 300), nn.ReLU(), nn.Dropout(p=0.31), nn.Linear(300, 20), nn.ReLU(), nn.Dropout(p=0.31))
model.apply(init_weights)

#Set loss
CEloss = nn.CrossEntropyLoss()

#CUDA related stuff...
if torch.cuda.is_available():
    model.cuda()
model.to(device)

#Set optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#Convert target to tensor
ytargets = LongTensor(ylab)    

In [7]:
# Start Training
for epoch in range(30):
    running_loss = 0.0
    measure_start = time.time()
    for i in range(X.shape[0]):
        #For Dropout
        model.train()
        
        #Get Input
        row = X.getrow(i)
        
        #Bug in numpy
        row = row.astype('float64')
        row = row.toarray()
        row = row.astype('float32')
        
        #Get Right Input Tensor
        input = torch.from_numpy(row).to(device)
            
        #Set target
        target = ytargets[i]
        
        #Put input into model to get output
        output = model(input)
        
        #calculate loss
        loss = CEloss(output, target)
        
        #backward step
        loss.backward()
        
        #optimizer step
        optimizer.step()
        #Reset optimizer grad
        optimizer.zero_grad()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            measure_end = time.time()
            print('[%d, %5d] loss: %.3f, time: %.3f seconds' %
                  (epoch + 1, i + 1, running_loss / 2000, measure_end-measure_start))
            running_loss = 0.0
            measure_start = time.time()

[1,  2000] loss: 2.614, time: 10.520 seconds
[1,  4000] loss: 2.380, time: 12.005 seconds
[1,  6000] loss: 2.339, time: 12.869 seconds
[1,  8000] loss: 2.195, time: 14.481 seconds
[1, 10000] loss: 2.202, time: 14.118 seconds
[1, 12000] loss: 2.205, time: 14.409 seconds
[1, 14000] loss: 2.213, time: 17.528 seconds
[1, 16000] loss: 2.110, time: 15.447 seconds
[1, 18000] loss: 2.131, time: 14.854 seconds
[1, 20000] loss: 2.075, time: 14.060 seconds
[1, 22000] loss: 2.071, time: 14.729 seconds
[1, 24000] loss: 2.130, time: 15.192 seconds
[1, 26000] loss: 2.143, time: 15.921 seconds
[1, 28000] loss: 2.013, time: 14.794 seconds
[1, 30000] loss: 2.062, time: 14.324 seconds
[1, 32000] loss: 2.017, time: 16.124 seconds
[1, 34000] loss: 2.032, time: 15.432 seconds
[1, 36000] loss: 2.072, time: 14.924 seconds
[1, 38000] loss: 2.040, time: 14.793 seconds
[2,  2000] loss: 2.012, time: 19.060 seconds
[2,  4000] loss: 2.002, time: 16.328 seconds
[2,  6000] loss: 1.985, time: 16.486 seconds
[2,  8000]

[10, 24000] loss: 1.795, time: 13.468 seconds
[10, 26000] loss: 1.850, time: 13.347 seconds
[10, 28000] loss: 1.824, time: 13.185 seconds
[10, 30000] loss: 1.873, time: 13.274 seconds
[10, 32000] loss: 1.818, time: 13.286 seconds
[10, 34000] loss: 1.828, time: 13.303 seconds
[10, 36000] loss: 1.853, time: 13.369 seconds
[10, 38000] loss: 1.835, time: 13.426 seconds
[11,  2000] loss: 1.847, time: 14.200 seconds
[11,  4000] loss: 1.852, time: 14.227 seconds
[11,  6000] loss: 1.840, time: 14.229 seconds
[11,  8000] loss: 1.797, time: 14.341 seconds
[11, 10000] loss: 1.804, time: 14.393 seconds
[11, 12000] loss: 1.779, time: 14.321 seconds
[11, 14000] loss: 1.823, time: 14.398 seconds
[11, 16000] loss: 1.819, time: 14.115 seconds
[11, 18000] loss: 1.822, time: 14.323 seconds
[11, 20000] loss: 1.832, time: 14.147 seconds
[11, 22000] loss: 1.847, time: 14.363 seconds
[11, 24000] loss: 1.843, time: 14.305 seconds
[11, 26000] loss: 1.876, time: 14.371 seconds
[11, 28000] loss: 1.803, time: 14.

[20,  2000] loss: 1.758, time: 13.373 seconds
[20,  4000] loss: 1.797, time: 13.654 seconds
[20,  6000] loss: 1.798, time: 13.090 seconds
[20,  8000] loss: 1.730, time: 13.281 seconds
[20, 10000] loss: 1.816, time: 13.353 seconds
[20, 12000] loss: 1.762, time: 14.266 seconds
[20, 14000] loss: 1.785, time: 14.367 seconds
[20, 16000] loss: 1.816, time: 14.452 seconds
[20, 18000] loss: 1.800, time: 13.854 seconds
[20, 20000] loss: 1.744, time: 14.252 seconds
[20, 22000] loss: 1.766, time: 14.367 seconds
[20, 24000] loss: 1.779, time: 14.373 seconds
[20, 26000] loss: 1.731, time: 14.427 seconds
[20, 28000] loss: 1.769, time: 14.274 seconds
[20, 30000] loss: 1.751, time: 14.287 seconds
[20, 32000] loss: 1.760, time: 14.220 seconds
[20, 34000] loss: 1.715, time: 14.238 seconds
[20, 36000] loss: 1.772, time: 14.376 seconds
[20, 38000] loss: 1.739, time: 14.012 seconds
[21,  2000] loss: 1.793, time: 14.358 seconds
[21,  4000] loss: 1.754, time: 14.387 seconds
[21,  6000] loss: 1.765, time: 14.

[29, 18000] loss: 1.751, time: 16.512 seconds
[29, 20000] loss: 1.689, time: 16.458 seconds
[29, 22000] loss: 1.684, time: 17.466 seconds
[29, 24000] loss: 1.709, time: 16.327 seconds
[29, 26000] loss: 1.736, time: 16.159 seconds
[29, 28000] loss: 1.715, time: 17.181 seconds
[29, 30000] loss: 1.693, time: 15.364 seconds
[29, 32000] loss: 1.694, time: 14.712 seconds
[29, 34000] loss: 1.734, time: 14.622 seconds
[29, 36000] loss: 1.715, time: 16.261 seconds
[29, 38000] loss: 1.728, time: 15.683 seconds
[30,  2000] loss: 1.719, time: 14.853 seconds
[30,  4000] loss: 1.761, time: 14.580 seconds
[30,  6000] loss: 1.794, time: 14.654 seconds
[30,  8000] loss: 1.722, time: 14.851 seconds
[30, 10000] loss: 1.706, time: 14.923 seconds
[30, 12000] loss: 1.644, time: 14.533 seconds
[30, 14000] loss: 1.717, time: 15.317 seconds
[30, 16000] loss: 1.677, time: 14.042 seconds
[30, 18000] loss: 1.748, time: 13.673 seconds
[30, 20000] loss: 1.697, time: 16.285 seconds
[30, 22000] loss: 1.694, time: 147

In [8]:
path = "../models/pytorch_kaggle_cooking/pytorch_kaggle_cooking.pt"
torch.save(model, path)
model2 = torch.load(path)
print(model2.eval())


Sequential(
  (0): Linear(in_features=3010, out_features=300, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.31)
  (3): Linear(in_features=300, out_features=20, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.31)
)


NameError: name 'history' is not defined